In [1]:
import collections
import os
import tensorflow as tf
import pandas as pd

tf.__version__


'2.13.0'

In [2]:
train_dir = r'https://raw.githubusercontent.com/sedeba19/NLP--Random-Models/main/train.csv'
test_dir = r'https://raw.githubusercontent.com/sedeba19/NLP--Random-Models/main/test.csv'

In [3]:
train_dir

'https://raw.githubusercontent.com/sedeba19/NLP--Random-Models/main/train.csv'

In [4]:
# Make a train_df
train_df = pd.read_csv(train_dir)
test_df = pd.read_csv(test_dir)
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac = 1, random_state = 1)
train_df_shuffled.head()

,id,keyword,location,text,target
3228,4632,emergency%20services,"Sydney, New South Wales",Goulburn man Henry Van Bilsen missing: Emergen...,1
3706,5271,fear,NaN,The things we fear most in organizations--fluc...,0
6957,9982,tsunami,Land Of The Kings,@tsunami_esh ?? hey Esh,0
2887,4149,drown,NaN,@POTUS you until you drown by water entering t...,0
7464,10680,wounds,"cody, austin follows ?*?",Crawling in my skin\nThese wounds they will no...,1


In [6]:
train_df_shuffled.shape

(7613, 5)

In [7]:
# How many examples of each class?
train_df['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [8]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# Visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index: random_index +5].itertuples():
    _, text, target = row
    if target > 0:
        print(f"Target:{target}", "(real disaster)")
    else:
        print(f"Target:{target}", "(not a real disaster)")
    
    print(f"Text: \n{text}")
    print("-----\n")

Target:0 (not a real disaster)
Text: 
It's been raining since you left me
Now I'm drowning in the flood
You see I've always been a fighter
But without you I give up
-----

Target:0 (not a real disaster)
Text: 
Quality Metrics Penalties May Harm Patient Care PCPs Say: Primary care physicians generally hold positive vi... http://t.co/TdwprVB04y
-----

Target:0 (not a real disaster)
Text: 
China is only delaying the deluge: If the fundamentals of an economy do not support the valuations of a stock ... http://t.co/fwIkyUrC18
-----

Target:1 (real disaster)
Text: 
RÌ©union Debris Is Almost Surely From Flight 370 Officials Say - New York Times http://t.co/gyQLAOz3l2
-----

Target:0 (not a real disaster)
Text: 
@ryanoss123 No worries you'd have to be on every hitters most pitchers got destroyed
-----


### Split data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df['text'],
                                                                            train_df['target'],
                                                                            test_size = 0.1,
                                                                            random_state = 42)

In [11]:
train_df['text'].shape

(7613,)

In [12]:
train_df["text"]

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [13]:
train_df['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [14]:
train_df['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [15]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

### Converting text into numbers: Text Vectorization also known as Tokenization

In [16]:
# Get the total words
total_words = 0
for i in train_sentences:
    total_words += len(i.split())

total_words

101905

In [17]:
train_sentences.shape

(6851,)

In [18]:
# Get the average words per sentence or line
avg_words_per_sentence = round(total_words/len(train_sentences))
avg_words_per_sentence

15

In [19]:
# Normally, total number of words is equal or less than the max vocabulary length
max_vocab_length = 10000

# max length of a statement is equal or more than the average word per sentence or line
max_length = 15

In [20]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

# Fit the text vectorizer to the training set
text_vectorizer.adapt(train_sentences)

In [21]:
# Create a sample sentence and tokenize it
samp_sentence = "There's a flood in my street!"
print(samp_sentence, f"Sentence length is {len(samp_sentence)}.")
text_vectorizer([samp_sentence])
#len(tf.squeeze(text_vectorizer([samp_sentence])))

There's a flood in my street! Sentence length is 29.


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[282,   3, 206,   4,  13, 674,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [22]:
# Choose random sentence from the training dataset and tokenize it
rand_train_sentence = random.choice(train_sentences)
print(f"Original text: \n\n {rand_train_sentence} \n\nVectorized text:")
text_vectorizer([rand_train_sentence])

Original text: 

 After the green line derailment my concern for track that looks like this goes up a bit... @cta @CTAFails http://t.co/1uDz0NVOEH 

Vectorized text:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  38,    2, 1126,  624,  364,   13, 3084,   10, 1951,   16,  288,
          25,   19,  519,   29]], dtype=int64)>

In [23]:
# Get unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'with',
 'it',
 'that',
 'at',
 'by',
 'this',
 'from',
 'are',
 'be',
 'was',
 'have',
 'like',
 'as',
 'me',
 'but',
 'up',
 'just',
 'so',
 'im',
 'not',
 'amp',
 'your',
 'out',
 'all',
 'after',
 'its',
 'has',
 'no',
 'will',
 'an',
 'fire',
 'when',
 'if',
 'we',
 'get',
 'now',
 'more',
 'via',
 'new',
 'about',
 'what',
 'people',
 'or',
 'news',
 'he',
 'they',
 'over',
 'one',
 'been',
 'how',
 'dont',
 'who',
 'video',
 'into',
 'were',
 'do',
 'us',
 'can',
 'emergency',
 '2',
 'disaster',
 'there',
 'her',
 'some',
 'than',
 'still',
 'would',
 'his',
 'police',
 'crash',
 'burning',
 'suicide',
 'body',
 'california',
 'back',
 'time',
 'buildings',
 'had',
 'why',
 'off',
 'them',
 'got',
 'man',
 'see',
 'storm',
 'know',
 'going',
 'cant',
 'world',
 'first',
 'day',
 'rt',
 'nuclear',
 'love',
 'youtube',
 'our',
 'attack',
 'go',
 'fires',
 'two',
 'their',
 'bomb',

In [24]:
len(words_in_vocab)

10000

In [25]:
# Top 5 words in words_in_vocab
words_in_vocab[:5]

['', '[UNK]', 'the', 'a', 'in']

In [26]:
# Least 5 words in words_in_vocab
words_in_vocab[-5:]

['pakthey', 'pakistan\x89Ûªs', 'pakistans', 'pajamas', 'paints']

### Embedding layer

To make our embedding, we are going to use TensorFlow’s embedding layer: https://www.tensorflow.org/api/docs/python/tf/keras/layers/

The parameters are below mostly used.
* input_dim = size of the vocabulary
* output_dim = size of the output embedding vector.  For example, a value of 100 would mean each token gets represented by a vector 100 long
* input_length = length of the sequences being passed to the embedding layer

In [27]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 64,
                             embeddings_initializer = "uniform",
                             input_length = max_length)

In [28]:
# Get a random sentence from the training set and return Original text, Vectorized text, Embedded text
rand_sentence = random.choice(train_sentences)
print(f"Original text: {rand_sentence}\n")
print(f"Vectorized text: {text_vectorizer([rand_sentence])}\n")
print(f"Vectorized text shape: {text_vectorizer([rand_sentence]).shape}")
print(f"Embbedded text: {embedding(text_vectorizer([rand_sentence]))}")
print(f"Embedded text shape: {embedding(text_vectorizer([rand_sentence])).shape}")

Original text: @spinningbot Are you another Stand-user? If you are I will have to detonate you with my Killer Queen.

Vectorized text: [[3382   21   12  162 1977   46   12   21    8   42   24    5  440   12
    14]]

Vectorized text shape: (1, 15)
Embbedded text: [[[ 2.65137888e-02 -1.56797171e-02 -4.10049669e-02 -3.66419926e-02
   -3.48515622e-02 -3.02727949e-02  1.19347349e-02  3.15733440e-02
   -8.41468573e-03  4.84174378e-02  2.58437507e-02  1.05426423e-02
    4.93105538e-02 -6.10452890e-03 -8.53536278e-03 -2.54497770e-02
   -1.73393600e-02 -4.81537357e-02 -4.23475392e-02 -2.39766482e-02
    4.76606935e-03 -2.75098328e-02  2.03125142e-02  3.97751480e-03
   -1.06533654e-02 -2.02704221e-04  3.46028097e-02 -1.19080544e-02
    1.81269981e-02  2.11239196e-02 -1.06723532e-02 -3.89666669e-02
    2.41673626e-02  4.12966646e-02 -5.45893982e-03  1.22954957e-02
    1.87972225e-02 -2.76230108e-02  3.95550989e-02 -1.30771771e-02
    4.26854976e-02 -4.70588803e-02  3.74016203e-02 -3.30164917e-02

## Model_7: TensorFlow Hub Pretrained Feature Extractor (10% of Data)

In [29]:
# Create subsets of 10% of the training and label data
train_10_percent = train_df[['text', "target"]].sample(frac= 0.1, random_state=42)
train_sentences_train_10_percent = train_10_percent["text"].to_list()
train_labels_train_10_percent = train_10_percent["target"].to_list()

In [30]:
len(train_sentences_train_10_percent), len(train_labels_train_10_percent)

(761, 761)

In [31]:
# Check the number of targets in our subset of data
train_10_percent['target'].value_counts()

target
0    425
1    336
Name: count, dtype: int64

In [32]:
train_df['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [33]:
# Create a Keras Layer using the USE pretrained layer from tensorflow hub
import tensorflow_hub as hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape= [], # empty because it is variable input length (any length)
                                        dtype = tf.string,
                                        trainable= False,
                                        name = "USE")

In [34]:
# Create model using the Sequential API
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation = "relu"),
    layers.Dense(1, activation = "sigmoid")
], name = "model_7_USE_10_percent")

# Compile
model_7.compile(loss = 'binary_crossentropy',
                optimizer= tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

model_7.summary()


Model: "model_7_USE_10_percent"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [35]:
model_7_history = model_7.fit(train_sentences_train_10_percent,
                              train_labels_train_10_percent,
                              epochs = 5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
24/24 [==============================] - 2s 24ms/step - loss: 0.6730 - accuracy: 0.7109 - val_loss: 0.6363 - val_accuracy: 0.8241
Epoch 2/5
24/24 [==============================] - 0s 8ms/step - loss: 0.6034 - accuracy: 0.8173 - val_loss: 0.5569 - val_accuracy: 0.8241
Epoch 3/5
24/24 [==============================] - 0s 8ms/step - loss: 0.5246 - accuracy: 0.8187 - val_loss: 0.4845 - val_accuracy: 0.8281
Epoch 4/5
24/24 [==============================] - 0s 8ms/step - loss: 0.4638 - accuracy: 0.8160 - val_loss: 0.4323 - val_accuracy: 0.8241
Epoch 5/5
24/24 [==============================] - 0s 7ms/step - loss: 0.4210 - accuracy: 0.8200 - val_loss: 0.3980 - val_accuracy: 0.8320


In [36]:
# Make predictions with the model trained on 10% of the data
model_7_pred_probs = model_7.predict(val_sentences)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))

24/24 [==============================] - 0s 4ms/step


In [37]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_7_accuracy_score = accuracy_score(val_labels, model_7_preds)
model_7_precision, model_7_recall, model_7_f1score, _, =precision_recall_fscore_support(val_labels, model_7_preds, average = "weighted")

In [38]:
dict_result = {"model_7_accuracy_score": model_7_accuracy_score,
               "model_7_precision": model_7_precision,
               "model_7_recall": {model_7_recall},
               "model_7_f1score": {model_7_f1score}}
dict_result

{'model_7_accuracy_score': 0.8320209973753281,
 'model_7_precision': 0.8318192282313513,
 'model_7_recall': {0.8320209973753281},
 'model_7_f1score': {0.8314470735447851}}